# Task 1: Proportions, Confidence Intervals, and Mean/Proportion Differences

**Audience:** Undergraduate or early graduate students practicing inferential statistics with Python.  
**Libraries:** `numpy`, `scipy.stats`, `matplotlib.pyplot` (no seaborn).  
**Theme:** Hands-on exercises using generated data so results are reproducible.

> Tip: Run cells in order. Cells marked with **🧪 Exercise** include TODOs for you to complete.

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

np.random.seed(42)  # for reproducibility

# Helper: pretty print function
def print_ci(label, low, high, conf=0.95):
    print(f"{label} {int(conf*100)}% CI: [{low:.4f}, {high:.4f}]")

# Helper: two-proportion z-test (manual implementation using normal approximation)
def two_proportion_ztest(x1, n1, x2, n2, alternative="two-sided"):
    """
    Returns (z_stat, p_value) for H0: p1 == p2 using pooled proportion.
    alternative in {"two-sided", "larger", "smaller"} tests p1 vs p2.
    """
    p1_hat = x1 / n1
    p2_hat = x2 / n2
    p_pool = (x1 + x2) / (n1 + n2)
    se = np.sqrt(p_pool * (1 - p_pool) * (1/n1 + 1/n2))
    if se == 0:
        return np.nan, np.nan
    z = (p1_hat - p2_hat) / se
    if alternative == "two-sided":
        p = 2 * (1 - stats.norm.cdf(abs(z)))
    elif alternative == "larger":
        p = 1 - stats.norm.cdf(z)
    elif alternative == "smaller":
        p = stats.norm.cdf(z)
    else:
        raise ValueError("alternative must be 'two-sided', 'larger', or 'smaller'")
    return z, p


## 1) One-Sample Proportion Test and Confidence Interval

We'll simulate a Bernoulli process (e.g., **email click-through**: success = clicked).  
Suppose the true click rate is `p = 0.18`. We take a sample of size `n = 400`.

In [ ]:
# Generate Bernoulli data
p_true = 0.18
n = 400
data = stats.bernoulli.rvs(p_true, size=n)
x = data.sum()
phat = x / n

print(f"Sample size n = {n}, successes x = {x}, sample proportion = {phat:.4f}")


In [ ]:
# Hypothesis test: H0: p = 0.20 vs H1: p != 0.20 using exact binomial test
res = stats.binomtest(k=x, n=n, p=0.20, alternative='two-sided')
print("Exact binomial test against p0 = 0.20")
print(f"p-value = {res.pvalue:.6f}")
# Exact Clopper-Pearson CI via SciPy (equal-tailed)
ci_low, ci_high = res.proportion_ci(confidence_level=0.95, method="exact")
print_ci("Exact (Clopper-Pearson)", ci_low, ci_high)


In [ ]:
# Normal-approx (Wald) CI with continuity correction OFF (for teaching/contrast)
z = stats.norm.ppf(0.975)
se = np.sqrt(phat*(1-phat)/n)
wald_low = phat - z*se
wald_high = phat + z*se
print_ci("Normal approx (Wald)", wald_low, wald_high)


### Visual: Proportion with 95% CI

We'll plot the point estimate with its 95% CI. (One figure only.)

In [ ]:
# Plot: point estimate with error bar
plt.figure()
center = phat
err = (wald_high - wald_low)/2
plt.errorbar([0], [center], yerr=[err], fmt='o', capsize=8)
plt.xticks([0], ["Proportion"])
plt.title("Sample Proportion with 95% CI (Normal Approx)")
plt.ylabel("Proportion")
plt.ylim(0, 1)
plt.show()


**🧪 Exercise 1.1 — Change the null and re-test**  
Using the **exact** binomial test above, test `H0: p = 0.15` (two-sided).  
- Report the p-value.  
- Compute the 90% exact CI and compare its width to the 95% CI.

In [ ]:
# TODO: Implement the test and CI requested in Exercise 1.1
# Hints:
# - Use stats.binomtest with p=0.15
# - Use .proportion_ci(confidence_level=0.90, method="exact")
# Your code below:


## 2) Difference of Means (Two Independent Samples)

Imagine two versions of a landing page (A and B) produce **time-on-page** (in seconds).  
We assume approximately normal distributions with possibly different variances.

In [ ]:
# Generate two samples with slightly different means
nA, nB = 120, 140
muA, muB = 60, 65
sdA, sdB = 12, 16

A = np.random.normal(muA, sdA, size=nA)
B = np.random.normal(muB, sdB, size=nB)

print(f"Group A: n={nA}, mean={A.mean():.2f}, std={A.std(ddof=1):.2f}")
print(f"Group B: n={nB}, mean={B.mean():.2f}, std={B.std(ddof=1):.2f}")


In [ ]:
# Welch's t-test (does not assume equal variances)
t_stat, p_val = stats.ttest_ind(A, B, equal_var=False)
print("Welch's two-sample t-test (H0: mu_A == mu_B)")
print(f"t = {t_stat:.4f}, p-value = {p_val:.6f}")


### Visual: Sample Distributions

In [ ]:
# Plot histograms for A and B (each chart in its own figure)
plt.figure()
plt.hist(A, bins=20, alpha=0.7)
plt.title("Group A: Time on Page (seconds)")
plt.xlabel("Seconds"); plt.ylabel("Frequency")
plt.show()

plt.figure()
plt.hist(B, bins=20, alpha=0.7)
plt.title("Group B: Time on Page (seconds)")
plt.xlabel("Seconds"); plt.ylabel("Frequency")
plt.show()


**🧪 Exercise 2.1 — Equal-variance t-test and CI**  
1. Run the equal-variance (pooled) two-sample t-test on `A` and `B`.  
2. Construct a 95% CI for the difference in means `mu_A - mu_B` under the equal-variance assumption.

In [ ]:
# TODO: Implement Exercise 2.1
# Hints:
# - Use stats.ttest_ind(A, B, equal_var=True)
# - For CI, compute pooled variance Sp^2, standard error for difference, and use t critical value.
# Your code below:


## 3) Difference of Proportions (Two Samples)

Suppose we A/B test a signup flow.  
- In Group A, n1 users see the original flow, with x1 signups.  
- In Group B, n2 users see the new flow, with x2 signups.

We'll test H0: p1 = p2 using the pooled z-test.

In [ ]:
# Simulate two independent binomial samples
n1, n2 = 600, 650
p1_true, p2_true = 0.22, 0.27
x1 = stats.binom.rvs(n1, p1_true)
x2 = stats.binom.rvs(n2, p2_true)
p1_hat, p2_hat = x1/n1, x2/n2

print(f"Group A: x1={x1}, n1={n1}, p1_hat={p1_hat:.4f}")
print(f"Group B: x2={x2}, n2={n2}, p2_hat={p2_hat:.4f}")
z, p = two_proportion_ztest(x1, n1, x2, n2, alternative="two-sided")
print(f"Two-proportion z-test: z={z:.4f}, p-value={p:.6f}")


### Visual: Proportions with 95% CIs

We'll show both sample proportions with normal-approximation 95% CIs side by side.

In [ ]:
# Compute Wald CIs for each proportion
zcrit = stats.norm.ppf(0.975)
se1 = np.sqrt(p1_hat*(1-p1_hat)/n1)
se2 = np.sqrt(p2_hat*(1-p2_hat)/n2)
ci1 = (p1_hat - zcrit*se1, p1_hat + zcrit*se1)
ci2 = (p2_hat - zcrit*se2, p2_hat + zcrit*se2)

# Plot with error bars
plt.figure()
centers = [p1_hat, p2_hat]
errs = [(ci1[1]-ci1[0])/2, (ci2[1]-ci2[0])/2]
plt.errorbar([0,1], centers, yerr=errs, fmt='o', capsize=8)
plt.xticks([0,1], ["Group A", "Group B"])
plt.title("Sample Proportions with 95% Wald CIs")
plt.ylabel("Proportion")
plt.ylim(0,1)
plt.show()


**🧪 Exercise 3.1 — One-sided test**  
Test `H1: p2 > p1` (i.e., the new flow increases signup rate).  
- Compute the one-sided p-value using the helper function.  
- Interpret the result at α = 0.05.

In [ ]:
# TODO: Implement Exercise 3.1
# Hint: two_proportion_ztest(..., alternative="larger") tests p1 > p2.
# To test p2 > p1, you can swap inputs or adapt the logic.
# Your code below:


## 4) Practice: Generate Your Own Data

Create your own synthetic datasets and redo all analyses. Suggestions:

1. Proportions: Choose p and n, generate Bernoulli data.  
   - Test H0: p = p0 with stats.binomtest (two-sided and one-sided).  
   - Build 90%, 95%, and 99% exact CIs and compare widths.  
   - Visualize the point estimate with CI.

2. Difference of Means: Generate two normal samples with your chosen means/SDs.  
   - Run Welch's and equal-variance t-tests.  
   - Construct CIs for mu1 - mu2 under both assumptions.  
   - Visualize each group with a histogram.

3. Difference of Proportions: Choose n1, n2, p1, p2; generate binomial counts.  
   - Run the two-proportion z-test (two-sided and one-sided).  
   - Plot both sample proportions with 95% CIs.

In [ ]:
# 🧪 Exercise 4.1 — Your custom proportions experiment
# TODO: Set p, n, generate data, run tests and CIs, and plot.
# Your code below:


In [ ]:
# 🧪 Exercise 4.2 — Your custom two-mean experiment
# TODO: Generate two normal samples, test equality of means with both methods, compute CIs, plot histograms.
# Your code below:


In [ ]:
# 🧪 Exercise 4.3 — Your custom two-proportion experiment
# TODO: Choose n1, n2, p1, p2; generate counts; run z-tests (two-sided and one-sided); plot CIs.
# Your code below:


### ⭐ Stretch Goals
- Compare exact CI (Clopper–Pearson) vs normal approximation CI for small and large n.  
- Explore how CI width changes with n; make a plot of CI width vs n at fixed p.  
- Bootstrap the difference of means and compare the bootstrap CI to the t-based CI.